[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/ndp-ep-py/blob/main/docs/source/tutorials/bulk_resource_management.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/ndp-ep-py/main?filepath=docs%2Fsource%2Ftutorials%2Fbulk_resource_management.ipynb)

# Bulk Resource Management Tutorial

Welcome to the advanced tutorial on bulk resource management with the NDP EP Python client!

## 🎯 What You'll Learn

This tutorial demonstrates real-world scenarios for managing large numbers of resources:

- **🏢 Organization Management**: Create and manage organizations for bulk operations
- **📊 Bulk Service Registration**: Register 100 services efficiently
- **📁 Bulk Dataset Registration**: Register 100 datasets with proper organization
- **📈 Progress Monitoring**: Track operations and handle errors gracefully
- **✅ Verification**: Confirm all resources were created successfully
- **🧹 Complete Cleanup**: Remove all created resources to leave system clean

## 🔧 Use Cases

This tutorial is perfect for:
- **Data Migration**: Moving large datasets between systems
- **System Setup**: Initial configuration with multiple resources
- **Testing Environments**: Creating test data for development
- **Batch Operations**: Learning patterns for bulk data management

## ⚠️ Prerequisites

- Valid NDP EP API credentials (token)
- Write permissions on the target API instance
- Basic understanding of the NDP EP API (complete the getting started tutorial first)

## 🛡️ Safety Note

This tutorial creates and then deletes 200+ resources. It's designed to be safe and self-contained, but always test on non-production systems first.

In [5]:
# Install required libraries
!pip install ndp-ep

# Import required modules
import time
import getpass
from typing import List, Dict, Any, Optional
from ndp_ep import APIClient

print("✅ Libraries installed and imported successfully!")
print("📚 Ready to start bulk resource management tutorial")

✅ Libraries installed and imported successfully!
📚 Ready to start bulk resource management tutorial


## 1. 🔐 Secure Authentication Setup

First, let's configure the client with your credentials. Your token will be hidden during input for security.

In [6]:
# Interactive API configuration
print("🔧 API Configuration")
print("=" * 30)

# Get API base URL
api_url = input("Enter API base URL [http://localhost:8003]: ").strip()
if not api_url:
    api_url = "http://localhost:8003"

print(f"📡 API URL: {api_url}")

# Get API token securely
print("\n🔐 Authentication")
print("Please enter your API token (it will be hidden):")
api_token = getpass.getpass("API Token: ")

if not api_token.strip():
    raise ValueError("❌ API token is required for this tutorial")

print("✅ Credentials configured securely")

🔧 API Configuration
📡 API URL: http://localhost:8003

🔐 Authentication
Please enter your API token (it will be hidden):
✅ Credentials configured securely


In [7]:
# Initialize and test the API client
print("🚀 Initializing API Client...")

try:
    client = APIClient(base_url=api_url, token=api_token)
    
    # Test connection
    system_status = client.get_system_status()
    print("✅ API client initialized successfully")
    print(f"🌐 Connected to: {api_url}")
    print("🔑 Authentication verified")
    
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    print("💡 Please check your credentials and API URL")
    raise

🚀 Initializing API Client...
✅ API client initialized successfully
🌐 Connected to: http://localhost:8003
🔑 Authentication verified


## 2. 📋 Tutorial Configuration

Let's set up the parameters for our bulk operations.

In [8]:
# Configuration for bulk operations
SERVICES_COUNT = 100
DATASETS_COUNT = 100
DATASETS_ORG = "test_services"  # Custom org for datasets
PREFIX = "test_"  # Prefix for all resource names

print("📊 Bulk Operation Configuration")
print("=" * 35)
print(f"Services to create: {SERVICES_COUNT}")
print(f"Datasets to create: {DATASETS_COUNT}")
print(f"Datasets organization: {DATASETS_ORG}")
print(f"Name prefix: {PREFIX}")
print(f"Total resources: {SERVICES_COUNT + DATASETS_COUNT}")

# Storage for tracking created resources
created_services = []
created_datasets = []
operation_log = []

📊 Bulk Operation Configuration
Services to create: 100
Datasets to create: 100
Datasets organization: test_services
Name prefix: test_
Total resources: 200


## 3. 🏗️ Utility Functions

Let's create helper functions for our bulk operations.

In [ ]:
def log_operation(operation: str, resource_type: str, 
                  resource_name: str, success: bool, 
                  details: str = "") -> None:
    """
    Log an operation for tracking and debugging.
    
    Args:
        operation: Type of operation (create, delete, verify)
        resource_type: Type of resource (service, dataset, organization)
        resource_name: Name of the resource
        success: Whether the operation was successful
        details: Additional details or error messages
    """
    timestamp = time.strftime("%H:%M:%S")
    status = "✅" if success else "❌"
    
    log_entry = {
        "timestamp": timestamp,
        "operation": operation,
        "resource_type": resource_type,
        "resource_name": resource_name,
        "success": success,
        "details": details
    }
    
    operation_log.append(log_entry)
    print(f"{status} [{timestamp}] {operation.title()} {resource_type}: {resource_name}")
    
    if details and not success:
        print(f"   └─ Error: {details}")


def generate_service_data(index: int) -> Dict[str, Any]:
    """
    Generate service data for registration.
    
    Args:
        index: Service index number
        
    Returns:
        Service data dictionary
    """
    service_num = str(index).zfill(3)  # Pad with zeros: 001, 002, etc.
    
    return {
        "service_name": f"{PREFIX}service_{service_num}",
        "service_title": f"{PREFIX.title()}Service {service_num}",
        "owner_org": "services",
        "service_url": f"https://api.example.com/service_{service_num}",
        "service_type": "REST API",
        "notes": f"Bulk tutorial test service number {service_num}",
        "health_check_url": f"https://api.example.com/service_{service_num}/health",
        "documentation_url": f"https://docs.example.com/service_{service_num}"
    }


def generate_dataset_data(index: int) -> Dict[str, Any]:
    """
    Generate dataset data for registration.

    Args:
        index: Dataset index number
        
    Returns:
        Dataset data dictionary
    """
    dataset_num = str(index).zfill(3)  # Pad with zeros: 001, 002, etc.
    
    return {
        "name": f"{PREFIX}dataset_{dataset_num}",
        "title": f"{PREFIX.title()}Dataset {dataset_num}",
        "owner_org": DATASETS_ORG,
        "notes": f"Bulk tutorial test dataset number {dataset_num}",
        "tags": ["test", "bulk", "tutorial"],
        "license_id": "cc-by",
        "private": False
    }


def print_progress(current: int, total: int, 
                   operation: str = "Processing") -> None:
    """
    Print progress information.
    
    Args:
        current: Current item number
        total: Total items to process
        operation: Description of the operation
    """
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    
    bar = "█" * filled_length + "-" * (bar_length - filled_length)
    print(f"\r{operation}: |{bar}| {current}/{total} ({percentage:.1f}%)", end="")
    
    if current == total:
        print()  # New line when complete


print("🔧 Utility functions defined successfully")
print("📝 Ready for bulk operations")

🔧 Utility functions defined successfully
📝 Ready for bulk operations


## 4. 🏢 Organization Setup

Before creating datasets, we need to ensure the organization exists.

In [33]:
# Check if datasets organization exists, create if needed
print(f"🏢 Setting up organization: {DATASETS_ORG}")
print("=" * 40)

try:
    # Check existing organizations
    existing_orgs = client.list_organizations(server="local")
    
    if DATASETS_ORG in existing_orgs:
        print(f"✅ Organization '{DATASETS_ORG}' already exists")
        log_operation("verify", "organization", DATASETS_ORG, True, 
                     "Already exists")
    else:
        # Create the organization
        print(f"🔨 Creating organization '{DATASETS_ORG}'...")
        
        org_data = {
            "name": DATASETS_ORG,
            "title": "Test Services Organization",
            "description": "Organization for bulk tutorial test datasets"
        }
        
        result = client.register_organization(org_data, server="local")
        print(f"✅ Organization created with ID: {result.get('id', 'unknown')}")
        log_operation("create", "organization", DATASETS_ORG, True, 
                     f"ID: {result.get('id', 'unknown')}")

except Exception as e:
    print(f"❌ Failed to setup organization: {e}")
    log_operation("create", "organization", DATASETS_ORG, False, str(e))
    raise

print("\n🎯 Organization setup complete!")

🏢 Setting up organization: test_services
✅ Organization 'test_services' already exists
✅ [16:42:50] Verify organization: test_services

🎯 Organization setup complete!


## 5. 📊 Bulk Service Registration

Now let's register 100 services efficiently with progress tracking.

In [ ]:
# Bulk service registration
print(f"📊 Starting bulk service registration ({SERVICES_COUNT} services)")
print("=" * 55)

services_start_time = time.time()
successful_services = 0
failed_services = 0

for i in range(1, SERVICES_COUNT + 1):
    try:
        # Generate service data
        service_data = generate_service_data(i)
        service_name = service_data["service_name"]
        
        # Register the service
        result = client.register_service(service_data, server="local")
        
        # Track successful registration
        created_services.append({
            "name": service_name,
            "id": result.get("id"),
            "index": i
        })
        
        successful_services += 1
        log_operation("create", "service", service_name, True, 
                     f"ID: {result.get('id', 'unknown')}")
        
    except Exception as e:
        failed_services += 1
        service_name = f"{PREFIX}service_{str(i).zfill(3)}"
        log_operation("create", "service", service_name, False, str(e))
        print(f"   ⚠️  Failed to create service {i}: {e}")
    
    # Update progress every 10 services or at the end
    if i % 10 == 0 or i == SERVICES_COUNT:
        print_progress(i, SERVICES_COUNT, "Creating services")
        
    # Small delay to avoid overwhelming the API
    time.sleep(0.1)

services_duration = time.time() - services_start_time

print(f"\n📈 Service Registration Summary")
print(f"   ✅ Successful: {successful_services}/{SERVICES_COUNT}")
print(f"   ❌ Failed: {failed_services}/{SERVICES_COUNT}")
print(f"   ⏱️  Duration: {services_duration:.2f} seconds")
print(f"   📊 Rate: {successful_services/services_duration:.1f} services/second")

if failed_services > 0:
    print(f"\n⚠️  {failed_services} services failed to register. Check logs above for details.")
else:
    print(f"\n🎉 All {SERVICES_COUNT} services registered successfully!")

📊 Starting bulk service registration (100 services)
❌ [23:18:47] Create service: test_service_001
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 1: name 'SERVICES_ORG' is not defined
❌ [23:18:47] Create service: test_service_002
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 2: name 'SERVICES_ORG' is not defined
❌ [23:18:47] Create service: test_service_003
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 3: name 'SERVICES_ORG' is not defined
❌ [23:18:47] Create service: test_service_004
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 4: name 'SERVICES_ORG' is not defined
❌ [23:18:47] Create service: test_service_005
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 5: name 'SERVICES_ORG' is not defined
❌ [23:18:47] Create service: test_service_006
   └─ Error: name 'SERVICES_ORG' is not defined
   ⚠️  Failed to create service 6:

## 6. 📁 Bulk Dataset Registration

Next, let's register 100 datasets using the general dataset registration function.

In [14]:
# Bulk dataset registration
print(f"📁 Starting bulk dataset registration ({DATASETS_COUNT} datasets)")
print("=" * 55)

datasets_start_time = time.time()
successful_datasets = 0
failed_datasets = 0

for i in range(1, DATASETS_COUNT + 1):
    try:
        # Generate dataset data
        dataset_data = generate_dataset_data(i)
        dataset_name = dataset_data["name"]
        
        # Register the dataset
        result = client.register_general_dataset(dataset_data, server="local")
        
        # Track successful registration
        created_datasets.append({
            "name": dataset_name,
            "id": result.get("id"),
            "index": i
        })
        
        successful_datasets += 1
        log_operation("create", "dataset", dataset_name, True, 
                     f"ID: {result.get('id', 'unknown')}")
        
    except Exception as e:
        failed_datasets += 1
        dataset_name = f"{PREFIX}dataset_{str(i).zfill(3)}"
        log_operation("create", "dataset", dataset_name, False, str(e))
        print(f"   ⚠️  Failed to create dataset {i}: {e}")
    
    # Update progress every 10 datasets or at the end
    if i % 10 == 0 or i == DATASETS_COUNT:
        print_progress(i, DATASETS_COUNT, "Creating datasets")
        
    # Small delay to avoid overwhelming the API
    time.sleep(0.1)

datasets_duration = time.time() - datasets_start_time

print(f"\n📈 Dataset Registration Summary")
print(f"   ✅ Successful: {successful_datasets}/{DATASETS_COUNT}")
print(f"   ❌ Failed: {failed_datasets}/{DATASETS_COUNT}")
print(f"   ⏱️  Duration: {datasets_duration:.2f} seconds")
print(f"   📊 Rate: {successful_datasets/datasets_duration:.1f} datasets/second")

if failed_datasets > 0:
    print(f"\n⚠️  {failed_datasets} datasets failed to register. Check logs above for details.")
else:
    print(f"\n🎉 All {DATASETS_COUNT} datasets registered successfully!")

📁 Starting bulk dataset registration (100 datasets)
✅ [16:16:47] Create dataset: test_dataset_001
✅ [16:16:49] Create dataset: test_dataset_002
✅ [16:16:51] Create dataset: test_dataset_003
✅ [16:16:53] Create dataset: test_dataset_004
✅ [16:16:55] Create dataset: test_dataset_005
✅ [16:16:57] Create dataset: test_dataset_006
✅ [16:16:59] Create dataset: test_dataset_007
✅ [16:17:01] Create dataset: test_dataset_008
✅ [16:17:03] Create dataset: test_dataset_009
✅ [16:17:05] Create dataset: test_dataset_010
Creating datasets: |███---------------------------| 10/100 (10.0%)✅ [16:17:07] Create dataset: test_dataset_011
✅ [16:17:09] Create dataset: test_dataset_012
✅ [16:17:11] Create dataset: test_dataset_013
✅ [16:17:13] Create dataset: test_dataset_014
✅ [16:17:15] Create dataset: test_dataset_015
✅ [16:17:17] Create dataset: test_dataset_016
✅ [16:17:19] Create dataset: test_dataset_017
✅ [16:17:21] Create dataset: test_dataset_018
✅ [16:17:23] Create dataset: test_dataset_019
✅ [16:17

## 7. ✅ Verification and Monitoring

Let's verify that all resources were created successfully and get an overview of what we've accomplished.

In [29]:
# Overall summary
total_duration = services_duration + datasets_duration
total_successful = successful_services + successful_datasets
total_failed = failed_services + failed_datasets
total_attempted = SERVICES_COUNT + DATASETS_COUNT

print("📊 BULK OPERATIONS SUMMARY")
print("=" * 50)
print(f"Services:")
print(f"   ✅ Created: {successful_services}/{SERVICES_COUNT}")
print(f"   ❌ Failed:  {failed_services}/{SERVICES_COUNT}")
print(f"\nDatasets:")
print(f"   ✅ Created: {successful_datasets}/{DATASETS_COUNT}")
print(f"   ❌ Failed:  {failed_datasets}/{DATASETS_COUNT}")
print(f"\nOverall:")
print(f"   📦 Total resources: {total_successful}/{total_attempted}")
print(f"   📈 Success rate: {(total_successful/total_attempted)*100:.1f}%")
print(f"   ⏱️  Total duration: {total_duration:.2f} seconds")
print(f"   🚀 Average rate: {total_successful/total_duration:.1f} resources/second")

# Memory usage summary
print(f"\n💾 Tracking Status:")
print(f"   📋 Services tracked: {len(created_services)}")
print(f"   📋 Datasets tracked: {len(created_datasets)}")
print(f"   📝 Log entries: {len(operation_log)}")

📊 BULK OPERATIONS SUMMARY
Services:
   ✅ Created: 0/10
   ❌ Failed:  10/10

Datasets:
   ✅ Created: 100/10
   ❌ Failed:  0/10

Overall:
   📦 Total resources: 100/20
   📈 Success rate: 500.0%
   ⏱️  Total duration: 210.74 seconds
   🚀 Average rate: 0.5 resources/second

💾 Tracking Status:
   📋 Services tracked: 0
   📋 Datasets tracked: 0
   📝 Log entries: 11


In [ ]:
# Verification through search
print("\n🔍 Verification through API search...")
print("=" * 40)

try:
    # Search for our test services
    service_search = client.advanced_search({
        "owner_org":"services",
        "server": "local"
    })
    
    # Count services that match our naming pattern
    found_services = [
        s for s in service_search 
        if s.get("name", "").startswith(f"{PREFIX}service_")
    ]
    
    print(f"🔍 Found {len(found_services)} test services via search")
    
except Exception as e:
    print(f"⚠️  Service search failed: {e}")
    found_services = []

try:
    # Search for our test datasets
    dataset_search = client.advanced_search({
        "search_term": PREFIX.rstrip("_"),
        "filter_list": [f"owner_org:{DATASETS_ORG}"],
        "server": "local"
    })
    
    # Count datasets that match our naming pattern
    found_datasets = [
        d for d in dataset_search 
        if d.get("name", "").startswith(f"{PREFIX}dataset_")
    ]
    
    print(f"🔍 Found {len(found_datasets)} test datasets via search")
    
except Exception as e:
    print(f"⚠️  Dataset search failed: {e}")
    found_datasets = []

# Verification summary
print(f"\n✅ Verification Results:")
print(f"   Services: {len(found_services)} found vs {successful_services} created")
print(f"   Datasets: {len(found_datasets)} found vs {successful_datasets} created")

if len(found_services) == successful_services and len(found_datasets) == successful_datasets:
    print(f"\n🎉 Perfect! All created resources are discoverable via search.")
else:
    print(f"\n⚠️  Discrepancy detected. Some resources may not be immediately searchable.")
    print(f"    This can be normal due to indexing delays.")


🔍 Verification through API search...
🔍 Found 100 test services via search
🔍 Found 0 test datasets via search

✅ Verification Results:


## 8. 🧹 Complete Cleanup Process

Now let's clean up all the resources we created to leave the system in its original state. This demonstrates proper resource lifecycle management.

In [ ]:
# Confirmation before cleanup
print("🧹 CLEANUP CONFIRMATION")
print("=" * 30)
print(f"About to delete:")
print(f"   📊 {len(created_services)} services")
print(f"   📁 {len(created_datasets)} datasets")
print(f"   🏢 1 organization ({DATASETS_ORG})")
print(f"\nTotal: {len(created_services) + len(created_datasets) + 1} resources")

# Get user confirmation
confirmation = input("\nProceed with cleanup? (yes/no): ").strip().lower()

if confirmation != 'yes':
    print("🚫 Cleanup cancelled by user")
    print("💡 Resources remain in the system for manual cleanup")
else:
    print("\n✅ Proceeding with cleanup...")

🧹 CLEANUP CONFIRMATION
About to delete:

✅ Proceeding with cleanup...


In [ ]:
# Only proceed if user confirmed
if confirmation == 'yes':
    # Delete services
    print(f"\n🗑️  Deleting services ({len(created_services)} services)")
    print("=" * 45)
    
    services_cleanup_start = time.time()
    deleted_services = 0
    failed_service_deletions = 0
    
    for i, service in enumerate(created_services, 1):
        try:
            # Delete by name (most reliable method)
            client.delete_resource_by_name(service["name"], server="local")
            deleted_services += 1
            log_operation("delete", "service", service["name"], True, 
                         f"Index: {service['index']}")
            
        except Exception as e:
            failed_service_deletions += 1
            log_operation("delete", "service", service["name"], False, str(e))
            print(f"   ⚠️  Failed to delete service {service['name']}: {e}")
        
        # Update progress
        if i % 10 == 0 or i == len(created_services):
            print_progress(i, len(created_services), "Deleting services")
            
        # Small delay
        time.sleep(0.1)
    
    services_cleanup_duration = time.time() - services_cleanup_start
    
    print(f"\n📈 Service Deletion Summary")
    print(f"   ✅ Deleted: {deleted_services}/{len(created_services)}")
    print(f"   ❌ Failed: {failed_service_deletions}/{len(created_services)}")
    print(f"   ⏱️  Duration: {services_cleanup_duration:.2f} seconds")
else:
    print("\n⏭️  Skipping service deletion (cleanup cancelled)")


📈 Service Deletion Summary
   ✅ Deleted: 0/0
   ❌ Failed: 0/0
   ⏱️  Duration: 0.00 seconds


In [19]:
# Only proceed if user confirmed
if confirmation == 'yes':
    # Delete datasets
    print(f"\n🗑️  Deleting datasets ({len(created_datasets)} datasets)")
    print("=" * 45)
    
    datasets_cleanup_start = time.time()
    deleted_datasets = 0
    failed_dataset_deletions = 0
    
    for i, dataset in enumerate(created_datasets, 1):
        try:
            # Delete by name (most reliable method)
            client.delete_resource_by_name(dataset["name"], server="local")
            deleted_datasets += 1
            log_operation("delete", "dataset", dataset["name"], True, 
                         f"Index: {dataset['index']}")
            
        except Exception as e:
            failed_dataset_deletions += 1
            log_operation("delete", "dataset", dataset["name"], False, str(e))
            print(f"   ⚠️  Failed to delete dataset {dataset['name']}: {e}")
        
        # Update progress
        if i % 10 == 0 or i == len(created_datasets):
            print_progress(i, len(created_datasets), "Deleting datasets")
            
        # Small delay
        time.sleep(0.1)
    
    datasets_cleanup_duration = time.time() - datasets_cleanup_start
    
    print(f"\n📈 Dataset Deletion Summary")
    print(f"   ✅ Deleted: {deleted_datasets}/{len(created_datasets)}")
    print(f"   ❌ Failed: {failed_dataset_deletions}/{len(created_datasets)}")
    print(f"   ⏱️  Duration: {datasets_cleanup_duration:.2f} seconds")
else:
    print("\n⏭️  Skipping dataset deletion (cleanup cancelled)")


🗑️  Deleting datasets (100 datasets)
✅ [16:26:58] Delete dataset: test_dataset_001
✅ [16:26:58] Delete dataset: test_dataset_002
✅ [16:26:59] Delete dataset: test_dataset_003
✅ [16:26:59] Delete dataset: test_dataset_004
✅ [16:27:00] Delete dataset: test_dataset_005
✅ [16:27:00] Delete dataset: test_dataset_006
✅ [16:27:01] Delete dataset: test_dataset_007
✅ [16:27:01] Delete dataset: test_dataset_008
✅ [16:27:01] Delete dataset: test_dataset_009
✅ [16:27:02] Delete dataset: test_dataset_010
Deleting datasets: |███---------------------------| 10/100 (10.0%)✅ [16:27:02] Delete dataset: test_dataset_011
✅ [16:27:02] Delete dataset: test_dataset_012
✅ [16:27:03] Delete dataset: test_dataset_013
✅ [16:27:03] Delete dataset: test_dataset_014
✅ [16:27:04] Delete dataset: test_dataset_015
✅ [16:27:04] Delete dataset: test_dataset_016
✅ [16:27:04] Delete dataset: test_dataset_017
✅ [16:27:05] Delete dataset: test_dataset_018
✅ [16:27:05] Delete dataset: test_dataset_019
✅ [16:27:05] Delete da

In [ ]:
# Only proceed if user confirmed
if confirmation == 'yes':
    # Delete the test organization (if we created it)
    print(f"\n🏢 Deleting organization: {DATASETS_ORG}")
    print("=" * 35)
    
    try:
        # Check if organization still exists and has no datasets
        remaining_datasets = client.advanced_search({
            "filter_list": [f"owner_org:{DATASETS_ORG}"],
            "server": "local"
        })
        
        if len(remaining_datasets) == 0:
            # Safe to delete organization
            client.delete_organization(DATASETS_ORG, server="local")
            print(f"✅ Organization '{DATASETS_ORG}' deleted successfully")
            log_operation("delete", "organization", DATASETS_ORG, True, 
                         "Cleanup complete")
        else:
            print(f"⚠️  Organization '{DATASETS_ORG}' still contains {len(remaining_datasets)} datasets")
            print(f"   Organization will not be deleted for safety")
            log_operation("delete", "organization", DATASETS_ORG, False, 
                         f"Still contains {len(remaining_datasets)} datasets")
            
    except Exception as e:
        print(f"❌ Failed to delete organization '{DATASETS_ORG}': {e}")
        log_operation("delete", "organization", DATASETS_ORG, False, str(e))
else:
    print("\n⏭️  Skipping organization deletion (cleanup cancelled)")


🏢 Deleting organization: test_services
✅ Organization 'test_services' deleted successfully
✅ [16:27:39] Delete organization: test_services


## 9. 📊 Final Verification and Tutorial Summary

Let's verify the cleanup was successful and summarize what we've learned.

In [30]:
# Final verification
print("🔍 FINAL VERIFICATION")
print("=" * 30)

if confirmation == 'yes':
    # Check if any test resources remain
    try:
        remaining_services = client.advanced_search({
            "search_term": PREFIX.rstrip("_"),
            "filter_list": [f"owner_org:{SERVICES_ORG}"],
            "server": "local"
        })
        
        remaining_test_services = [
            s for s in remaining_services 
            if s.get("name", "").startswith(f"{PREFIX}service_")
        ]
        
        print(f"🔍 Remaining test services: {len(remaining_test_services)}")
        
    except Exception as e:
        print(f"⚠️  Could not verify services cleanup: {e}")
    
    try:
        remaining_datasets = client.advanced_search({
            "search_term": PREFIX.rstrip("_"),
            "filter_list": [f"owner_org:{DATASETS_ORG}"],
            "server": "local"
        })
        
        remaining_test_datasets = [
            d for d in remaining_datasets 
            if d.get("name", "").startswith(f"{PREFIX}dataset_")
        ]
        
        print(f"🔍 Remaining test datasets: {len(remaining_test_datasets)}")
        
    except Exception as e:
        print(f"⚠️  Could not verify datasets cleanup: {e}")
    
    # Check organization
    try:
        orgs = client.list_organizations(server="local")
        org_exists = DATASETS_ORG in orgs
        print(f"🏢 Organization '{DATASETS_ORG}' exists: {org_exists}")
        
    except Exception as e:
        print(f"⚠️  Could not verify organization cleanup: {e}")

else:
    print("⏭️  Verification skipped (cleanup was cancelled)")
    print(f"💡 {len(created_services) + len(created_datasets)} resources remain in system")

🔍 FINAL VERIFICATION
🔍 Remaining test services: 0
🔍 Remaining test datasets: 0
🏢 Organization 'test_services' exists: True
